Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [26]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [27]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [28]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [29]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [30]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  n_hidden = 1024
  weights_h = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden]))
  biases_h = tf.Variable(tf.zeros([n_hidden]))
    
  weights_o = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  biases_o = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h) + biases_h)
  
  logits = tf.matmul(hidden, weights_o) + biases_o

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(weights_h) + tf.nn.l2_loss(biases_h) +
                  tf.nn.l2_loss(weights_o) + tf.nn.l2_loss(biases_o))
  # Add the regularization term to the loss.
  #loss += 5e-4 * regularizers
  loss += 1e-2 * regularizers
    
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h) + biases_h)  
  valid_logits = tf.matmul(valid_hidden, weights_o) + biases_o
  valid_prediction = tf.nn.softmax(valid_logits)

  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weights_h) + biases_h)
  test_logits = tf.matmul(test_hidden, weights_o) + biases_o
  test_prediction = tf.nn.softmax(test_logits)

In [31]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3515.323242
Minibatch accuracy: 12.5%
Validation accuracy: 37.7%
Minibatch loss at step 500: 21.057234
Minibatch accuracy: 91.4%
Validation accuracy: 84.8%
Minibatch loss at step 1000: 0.911430
Minibatch accuracy: 79.7%
Validation accuracy: 83.5%
Minibatch loss at step 1500: 0.856071
Minibatch accuracy: 78.1%
Validation accuracy: 83.7%
Minibatch loss at step 2000: 0.769579
Minibatch accuracy: 83.6%
Validation accuracy: 83.5%
Minibatch loss at step 2500: 0.714773
Minibatch accuracy: 83.6%
Validation accuracy: 83.8%
Minibatch loss at step 3000: 0.689601
Minibatch accuracy: 85.9%
Validation accuracy: 83.1%
Test accuracy: 89.5%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [35]:
train_dataset_2 = train_dataset[0:1000]
train_labels_2 = train_labels[0:1000]
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  n_hidden = 1024
  weights_h = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden]))
  biases_h = tf.Variable(tf.zeros([n_hidden]))
    
  weights_o = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  biases_o = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h) + biases_h)
    
  logits = tf.matmul(hidden, weights_o) + biases_o

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(weights_h) + tf.nn.l2_loss(biases_h) +
                  tf.nn.l2_loss(weights_o) + tf.nn.l2_loss(biases_o))
  # Add the regularization term to the loss.
  #loss += 5e-4 * regularizers
  loss += 1e-2 * regularizers
    
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h) + biases_h)  
  valid_logits = tf.matmul(valid_hidden, weights_o) + biases_o
  valid_prediction = tf.nn.softmax(valid_logits)

  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weights_h) + biases_h)
  test_logits = tf.matmul(test_hidden, weights_o) + biases_o
  test_prediction = tf.nn.softmax(test_logits)

# run
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels_2.shape[0] - batch_size)
    batch_data = train_dataset_2[offset:(offset + batch_size), :]
    batch_labels = train_labels_2[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3467.244629
Minibatch accuracy: 8.6%
Validation accuracy: 22.3%
Minibatch loss at step 500: 21.062040
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 1000: 0.524448
Minibatch accuracy: 100.0%
Validation accuracy: 81.2%
Minibatch loss at step 1500: 0.374651
Minibatch accuracy: 100.0%
Validation accuracy: 80.8%
Minibatch loss at step 2000: 0.357482
Minibatch accuracy: 100.0%
Validation accuracy: 80.7%
Minibatch loss at step 2500: 0.431197
Minibatch accuracy: 97.7%
Validation accuracy: 80.0%
Minibatch loss at step 3000: 0.355980
Minibatch accuracy: 100.0%
Validation accuracy: 80.6%
Test accuracy: 87.1%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [37]:
train_dataset_2 = train_dataset[0:1000]
train_labels_2 = train_labels[0:1000]
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  n_hidden = 1024
  weights_h = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden]))
  biases_h = tf.Variable(tf.zeros([n_hidden]))
    
  weights_o = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  biases_o = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h) + biases_h)
  hidden = tf.nn.dropout(hidden, 0.5)
    
  logits = tf.matmul(hidden, weights_o) + biases_o

# compute the loss
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(weights_h) + tf.nn.l2_loss(biases_h) +
                  tf.nn.l2_loss(weights_o) + tf.nn.l2_loss(biases_o))
  # Add the regularization term to the loss.  
  loss += 1e-2 * regularizers #loss += 5e-4 * regularizers
    
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  #  train_prediction = tf.nn.softmax(logits)
  # valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h) + biases_h)  
  # valid_logits = tf.matmul(valid_hidden, weights_o) + biases_o
  # valid_prediction = tf.nn.softmax(valid_logits)


  # test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weights_h) + biases_h)
  # test_logits = tf.matmul(test_hidden, weights_o) + biases_o
  # test_prediction = tf.nn.softmax(test_logits)
    
train_prediction = tf.nn.softmax(tf.matmul(tf.nn.dropout(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h) + biases_h), 0.5), weights_o) + biases_o)
valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h) + biases_h), weights_o) + biases_o)
test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h) + biases_h), weights_o) + biases_o)




# run
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels_2.shape[0] - batch_size)
    batch_data = train_dataset_2[offset:(offset + batch_size), :]
    batch_labels = train_labels_2[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3615.925049
Minibatch accuracy: 0.0%
Validation accuracy: 35.6%


/home/razvan/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:3: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  app.launch_new_instance()


Minibatch loss at step 500: 21.148340
Minibatch accuracy: 0.0%
Validation accuracy: 79.6%
Minibatch loss at step 1000: 0.540888
Minibatch accuracy: 0.0%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 0.424818
Minibatch accuracy: 0.0%
Validation accuracy: 80.6%
Minibatch loss at step 2000: 0.388451
Minibatch accuracy: 0.0%
Validation accuracy: 80.7%
Minibatch loss at step 2500: 0.472064
Minibatch accuracy: 0.0%
Validation accuracy: 79.8%
Minibatch loss at step 3000: 0.383910
Minibatch accuracy: 0.0%
Validation accuracy: 80.8%
Test accuracy: 9.7%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [ ]:
batch_size = 128
hidden_layer1_size = 1024
hidden_layer2_size = 105
use_multilayers = True

regularization_meta=0.03

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    keep_prob = tf.placeholder(tf.float32)

    weights_layer1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer1_size], stddev=0.03))
    biases_layer1 = tf.Variable(tf.zeros([hidden_layer1_size]))

    if use_multilayers:
        weights_layer2 = tf.Variable(tf.truncated_normal([hidden_layer1_size, hidden_layer1_size], stddev=0.01))
        biases_layer2 = tf.Variable(tf.zeros([hidden_layer1_size]))

        weights_layer3 = tf.Variable(tf.truncated_normal([hidden_layer1_size, hidden_layer2_size], stddev=0.01))
        biases_layer3 = tf.Variable(tf.zeros([hidden_layer2_size]))

        weights = tf.Variable(tf.truncated_normal([hidden_layer2_size if use_multilayers else hidden_layer1_size, num_labels], stddev=0.03))
        biases = tf.Variable(tf.zeros([num_labels]))

    # get the NN models
    def getNN3Layer(dSet, use_dropout):
        input_to_layer1 = tf.matmul(dSet, weights_layer1) + biases_layer1
        hidden_layer1_output = tf.nn.relu(input_to_layer1)

        logits_hidden1 = None
        if use_dropout:
            dropout_hidden1 = tf.nn.dropout(hidden_layer1_output, keep_prob)
            logits_hidden1 = tf.matmul(dropout_hidden1, weights_layer2) + biases_layer2
        else:
            logits_hidden1 = tf.matmul(hidden_layer1_output, weights_layer2) + biases_layer2

        hidden_layer2_output = tf.nn.relu(logits_hidden1)

        logits_hidden2 = None
        if use_dropout:
            dropout_hidden2 = tf.nn.dropout(hidden_layer2_output, keep_prob)
            logits_hidden2 = tf.matmul(dropout_hidden2, weights_layer3) + biases_layer3
        else:
            logits_hidden2 = tf.matmul(hidden_layer2_output, weights_layer3) + biases_layer3


        hidden_layer3_output = tf.nn.relu(logits_hidden2)
        logits = None
        if use_dropout:
            dropout_hidden3 = tf.nn.dropout(hidden_layer3_output, keep_prob)
            logits = tf.matmul(dropout_hidden3, weights) + biases
        else:
            logits = tf.matmul(hidden_layer3_output, weights) + biases

        return logits

    # get the NN models
    def getNN1Layer(dSet, use_dropout, w1, b1, w, b):
        input_to_layer1 = tf.matmul(dSet, w1) + b1
        hidden_layer1_output = tf.nn.relu(input_to_layer1)

        logits = None
        if use_dropout:
            dropout_hidden1 = tf.nn.dropout(hidden_layer1_output, keep_prob)
            logits = tf.matmul(dropout_hidden1, w) + b
        else:
            logits = tf.matmul(hidden_layer1_output, w) + b

        return logits

    # Training computation.
    logits = getNN3Layer(tf_train_dataset, True)
    logits_valid = getNN3Layer(tf_valid_dataset, True)
    logits_test = getNN3Layer(tf_test_dataset, True)

    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    #loss_l2 = loss + (regularization_meta * (tf.nn.l2_loss(weights)))

    global_step = tf.Variable(0) # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.3, global_step, 3500, 0.86, staircase=True)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

# Predictions for the training, validation, and test data.
train_prediction = tf.nn.softmax(logits)
valid_prediction = tf.nn.softmax(logits_valid)
test_prediction = tf.nn.softmax(logits_test)

num_steps = 12001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in xrange(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:0.5}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step", step, ":", l)
          print("Minibatch accuracy: %.1f%%" % accuracy(train_prediction.eval(feed_dict={tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:1.0}), batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(feed_dict={keep_prob:1.0}), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob:1.0}), test_labels))

Initialized
Minibatch loss at step 0 : 2.3032
Minibatch accuracy: 12.5%
Validation accuracy: 13.0%
Test accuracy: 12.9%
Test accuracy: 13.0%
Test accuracy: 10.0%
Test accuracy: 10.0%
Test accuracy: 10.0%
Test accuracy: 10.0%
Test accuracy: 22.9%
Test accuracy: 18.0%
Test accuracy: 15.0%
Test accuracy: 17.3%
Test accuracy: 24.5%
Test accuracy: 14.5%
Test accuracy: 25.3%
Test accuracy: 23.3%
Test accuracy: 24.3%
Test accuracy: 19.3%
Test accuracy: 23.9%
Test accuracy: 20.0%
Test accuracy: 12.0%
Test accuracy: 12.3%
Test accuracy: 14.1%
Test accuracy: 24.9%
Test accuracy: 14.3%
Test accuracy: 16.1%
Test accuracy: 22.2%
Test accuracy: 22.6%
Test accuracy: 34.9%
Test accuracy: 24.3%
Test accuracy: 31.2%
Test accuracy: 27.7%
Test accuracy: 26.9%
Test accuracy: 28.4%
Test accuracy: 32.0%
Test accuracy: 41.8%
Test accuracy: 41.1%
Test accuracy: 35.9%
Test accuracy: 30.9%
Test accuracy: 42.0%
Test accuracy: 52.6%
Test accuracy: 35.2%
Test accuracy: 40.2%
Test accuracy: 50.6%
Test accuracy: 47.7